# determining infection statuses

In [2]:
import pandas as pd

In [3]:
df = pd.read_pickle('/mnt/SYNO/macrohet_syno/results/dfs/trimmed_df.pkl')

In [4]:
df

,ID,Time (hours),Mtb Area (µm),dMtb Area (µm),Mphi Area (µm),dMphi Area (µm),Infection Status,Initial Infection Status,Final Infection Status,x,...,Acquisition ID,Experiment ID,Unique ID,Edge Status,Uptake,dMtb Area between frames (µm),Mtb Area Processed (µm),Time Model (hours),Mtb Area Model (µm),mtb_origin
0,1.3.12.ND0003,0.0,0.000000,0.000000,1400.533422,-557.437690,False,False,False,656.800842,...,"(3, 12)",ND0003,1.3.12,False,False,NaN,NaN,NaN,NaN,None
1,1.3.12.ND0003,0.5,0.000000,0.000000,1314.022525,-557.437690,False,False,False,661.850525,...,"(3, 12)",ND0003,1.3.12,False,False,0.000000,NaN,NaN,NaN,None
2,1.3.12.ND0003,1.0,0.000000,0.000000,1107.522736,-557.437690,False,False,False,664.685547,...,"(3, 12)",ND0003,1.3.12,False,False,0.000000,NaN,NaN,NaN,None
3,1.3.12.ND0003,1.5,0.000000,0.000000,1481.837127,-557.437690,False,False,False,668.500366,...,"(3, 12)",ND0003,1.3.12,False,False,0.000000,NaN,NaN,NaN,None
4,1.3.12.ND0003,2.0,NaN,0.000000,NaN,-557.437690,False,False,False,671.554077,...,"(3, 12)",ND0003,1.3.12,False,False,NaN,NaN,0.0,0.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1312269,998.5.5.ND0003,48.0,0.000000,-0.111742,813.037051,-539.871799,False,False,False,868.403076,...,"(5, 5)",ND0003,998.5.5,False,False,0.000000,0.0,NaN,NaN,None
1312270,998.5.5.ND0003,48.5,0.000000,-0.111742,745.745831,-539.871799,False,False,False,868.194641,...,"(5, 5)",ND0003,998.5.5,False,False,0.000000,0.0,NaN,NaN,None
1312271,998.5.5.ND0003,49.0,0.000000,-0.111742,668.219016,-539.871799,False,False,False,872.108582,...,"(5, 5)",ND0003,998.5.5,False,False,0.000000,0.0,NaN,NaN,None
1312272,998.5.5.ND0003,49.5,0.000000,-0.111742,662.609552,-539.871799,False,False,False,871.521606,...,"(5, 5)",ND0003,998.5.5,False,False,0.000000,0.0,NaN,NaN,None


In [5]:
def process_group(group):
    # Sort the group by 'Time (hours)' just in case
    group = group.sort_values(by='Time (hours)')
    
    # Filter out rows where 'Mtb Area (µm)' is NaN
    valid_group = group.dropna(subset=['Mtb Area Model (µm)'])
    
    # Determine the time range for the first three hours of non-NaN data
    initial_period = valid_group[valid_group['Time Model (hours)'] <= valid_group['Time Model (hours)'].min() + 3]
    
    # Determine the time range for the last three hours of non-NaN data
    final_period = valid_group[valid_group['Time Model (hours)'] >= valid_group['Time Model (hours)'].max() - 3]
    
    # Infection Status: If the mean Mtb Area (µm) in the entire group is >= 1.92
    group['Infection Status'] = valid_group['Mtb Area Model (µm)'] >= 1.92
    
    # Initial Infection Status: If the mean Mtb Area (µm) in the first 3 non-NaN hours is >= 1.92
    group['Initial Infection Status'] = (initial_period['Mtb Area Model (µm)'] >= 1.92).all()
    
    # Final Infection Status: If the mean Mtb Area (µm) in the last 3 non-NaN hours is >= 1.92
    group['Final Infection Status'] = (final_period['Mtb Area Model (µm)'] >= 1.92).all()

    ### maybe include a check if other frames are zero...
    
    return group

# Apply the function to each group
df = df.groupby('ID').apply(process_group)

/tmp/ipykernel_1584567/1134006858.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('ID').apply(process_group)


In [6]:
df

ID  Time (hours)  Mtb Area (µm)  \
ID                                                                    
1.3.12.ND0003  0         1.3.12.ND0003           0.0       0.000000   
               1         1.3.12.ND0003           0.5       0.000000   
               2         1.3.12.ND0003           1.0       0.000000   
               3         1.3.12.ND0003           1.5       0.000000   
               4         1.3.12.ND0003           2.0            NaN   
...                                ...           ...            ...   
998.5.5.ND0003 1312269  998.5.5.ND0003          48.0       0.000000   
               1312270  998.5.5.ND0003          48.5       0.000000   
               1312271  998.5.5.ND0003          49.0       0.000000   
               1312272  998.5.5.ND0003          49.5       0.000000   
               1312273  998.5.5.ND0003          50.0       0.156439   

                        dMtb Area (µm)  Mphi Area (µm)  dMphi Area (µm)  \
ID                                                                        
1.3.12.ND0003  0              0.000000     1400.533422      -557.437690   
               1              0.000000     1314.022525      -557.437690   
               2              0.000000     1107.522736      -557.437690   
               3              0.000000     1481.837127      -557.437690   
               4              0.000000             NaN      -557.437690   
...                                ...             ...              ...   
998.5.5.ND0003 1312269       -0.111742      813.037051      -539.871799   
               1312270       -0.111742      745.745831      -539.871799   
               1312271       -0.111742      668.219016      -539.871799   
               1312272       -0.111742      662.609552      -539.871799   
               1312273       -0.111742      642.987603      -539.871799   

                       Infection Status  Initial Infection Status  \
ID                                                                  
1.3.12.ND0003  0                    NaN                     False   
               1                    NaN                     False   
               2                    NaN                     False   
               3                    NaN                     False   
               4                  False                     False   
...                                 ...                       ...   
998.5.5.ND0003 1312269              NaN                     False   
               1312270              NaN                     False   
               1312271              NaN                     False   
               1312272              NaN                     False   
               1312273              NaN                     False   

                        Final Infection Status           x  ...  \
ID                                                          ...   
1.3.12.ND0003  0                         False  656.800842  ...   
               1                         False  661.850525  ...   
               2                         False  664.685547  ...   
               3                         False  668.500366  ...   
               4                         False  671.554077  ...   
...                                        ...         ...  ...   
998.5.5.ND0003 1312269                   False  868.403076  ...   
               1312270                   False  868.194641  ...   
               1312271                   False  872.108582  ...   
               1312272                   False  871.521606  ...   
               1312273                   False  871.464844  ...   

                        Acquisition ID  Experiment ID  Unique ID  Edge Status  \
ID                                                                              
1.3.12.ND0003  0               (3, 12)         ND0003     1.3.12        False   
               1               (3, 12)         ND0003     1.3.12        False   
               2               (3, 12)    

In [7]:
df.to_pickle('/mnt/SYNO/macrohet_syno/results/dfs/trimmed_df.pkl')

In [8]:
df[df['ID']=='2385.4.3.ND0002']

ID  Time (hours)  Mtb Area (µm)  \
ID                                                                     
2385.4.3.ND0002 533598  2385.4.3.ND0002          36.5     128.660096   
                533599  2385.4.3.ND0002          37.0     105.104817   
                533600  2385.4.3.ND0002          37.5     152.349466   
                533601  2385.4.3.ND0002          38.0     136.750240   
                533602  2385.4.3.ND0002          38.5            NaN   
...                                 ...           ...            ...   
                533667  2385.4.3.ND0002          71.5            NaN   
                533668  2385.4.3.ND0002          72.0     133.174486   
                533669  2385.4.3.ND0002          72.5     137.800617   
                533670  2385.4.3.ND0002          73.0     117.687001   
                533671  2385.4.3.ND0002          73.5     127.900249   

                        dMtb Area (µm)  Mphi Area (µm)  dMphi Area (µm)  \
ID                                                                        
2385.4.3.ND0002 533598       -0.759848     1102.673119       -83.248021   
                533599       -0.759848     1462.282223       -83.248021   
                533600       -0.759848     2550.339448       -83.248021   
                533601       -0.759848     1719.892945       -83.248021   
                533602       -0.759848             NaN       -83.248021   
...                                ...             ...              ...   
                533667       -0.759848             NaN       -83.248021   
                533668       -0.759848      796.677976       -83.248021   
                533669       -0.759848      979.421351       -83.248021   
                533670       -0.759848      985.388390       -83.248021   
                533671       -0.759848     1019.425098       -83.248021   

                       Infection Status  Initial Infection Status  \
ID                                                                  
2385.4.3.ND0002 533598              NaN                      True   
                533599              NaN                      True   
                533600              NaN                      True   
                533601              NaN                      True   
                533602             True                      True   
...                                 ...                       ...   
                533667              NaN                      True   
                533668              NaN                      True   
                533669              NaN                      True   
                533670              NaN                      True   
                533671              NaN                      True   

                        Final Infection Status           x  ...  \
ID                                                          ...   
2385.4.3.ND0002 533598                    True  812.489075  ...   
                533599                    True  818.677490  ...   
                533600                    True  818.304321  ...   
                533601                    True  825.857300  ...   
                533602                    True  828.400146  ...   
...                                        ...         ...  ...   
                533667                    True  818.557373  ...   
                533668                    True  821.706848  ...   
                533669                    True  823.189331  ...   
                533670                    True  820.340576  ...   
                533671                    True  823.368347  ...   

                        Acquisition ID  Experiment ID  Unique ID  Edge Status  \
ID                                                                              
2385.4.3.ND0002 533598          (4, 3)         ND0002   2385.4.3        False   
                533599          (4, 3)         ND0002   2385.4.3        False   
                533600        

In [75]:
df[df['ID']=='2385.4.3.ND0002'].dropna(subset=['Mtb Area Model (µm)'])

ID  Time (hours)  Mtb Area (µm)  \
ID                                                                     
2385.4.3.ND0002 534705  2385.4.3.ND0002          38.5            NaN   
                534706  2385.4.3.ND0002          39.0            NaN   
                534707  2385.4.3.ND0002          39.5            NaN   
                534708  2385.4.3.ND0002          40.0     130.403276   
                534709  2385.4.3.ND0002          41.0     126.939265   
...                                 ...           ...            ...   
                534765  2385.4.3.ND0002          69.0     119.541924   
                534766  2385.4.3.ND0002          69.5     114.021854   
                534767  2385.4.3.ND0002          70.0     139.990767   
                534768  2385.4.3.ND0002          70.5     130.492670   
                534769  2385.4.3.ND0002          71.0     135.431680   

                        dMtb Area (µm)  Mphi Area (µm)  dMphi Area (µm)  \
ID                                                                        
2385.4.3.ND0002 534705       -0.759848             NaN       -83.248021   
                534706       -0.759848             NaN       -83.248021   
                534707       -0.759848             NaN       -83.248021   
                534708       -0.759848     3433.528321       -83.248021   
                534709       -0.759848     1775.741752       -83.248021   
...                                ...             ...              ...   
                534765       -0.759848     1112.171216       -83.248021   
                534766       -0.759848     1200.090066       -83.248021   
                534767       -0.759848     1199.285521       -83.248021   
                534768       -0.759848     1156.085944       -83.248021   
                534769       -0.759848     1071.832242       -83.248021   

                        Infection Status  Initial Infection Status  \
ID                                                                   
2385.4.3.ND0002 534705              True                     False   
                534706              True                     False   
                534707              True                     False   
                534708              True                     False   
                534709              True                     False   
...                                  ...                       ...   
                534765              True                     False   
                534766              True                     False   
                534767              True                     False   
                534768              True                     False   
                534769              True                     False   

                        Final Infection Status           x  ...  Unique ID  \
ID                                                          ...              
2385.4.3.ND0002 534705                    True  828.400146  ...   2385.4.3   
                534706                    True  830.942993  ...   2385.4.3   
                534707                    True  833.485840  ...   2385.4.3   
                534708                    True  836.954041  ...   2385.4.3   
                534709                    True  819.655212  ...   2385.4.3   
...                                        ...         ...  ...        ...   
                534765                    True  817.376282  ...   2385.4.3   
                534766                    True  814.516907  ...   2385.4.3   
                534767                    True  815.781067  ...   2385.4.3   
                534768                    True  817.533325  ...   2385.4.3   
                534769                    True  818.120789  ...   2385.4.3   

                        Edge Status  Uptake  dMtb Area between frames (µm)  \
ID                                                                           
2385.4.3.ND0002 534705        False   Fals